Train the SimCLR architecture with ResNet18 as encoder

In [1]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os

from utilities import *
from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU, MaxPool2d, AvgPool2d, Linear, Dropout, CrossEntropyLoss
from torch.optim import SGD, Adam, AdamW, lr_scheduler
import torchvision
from tqdm import tqdm

# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "./saved_models"

In [2]:
# Set seeds for reproducibility
# fix this
seed = 203
torch.manual_seed(seed)

In [3]:
dir_init = './doppler_traces/' #Directory of data
subdirs_init = 'S1a' #Subdirs for training

In [4]:
dataset_csi_train = create_training_set(dir_init=dir_init, subdirs_init=subdirs_init, aggregate=True)
dataset_csi_val = create_validation_set(dir_init=dir_init, subdirs_init=subdirs_init, aggregate=True)
dataset_csi_test = create_test_set(dir_init=dir_init, subdirs_init=subdirs_init, aggregate=True)

In [5]:
class SimCLR(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()

        # Base model f(.), remember to put output shape equal to hidden_dim * 4
        self.convnet = Inceptionv4(num_classes=4*hidden_dim)  # Output of last linear layer

        # The MLP for g(.) consists of Linear->ReLU->Linear
        # Note that the resnet18 model provided by torchvision already has a single linear output layer which you can access with convnet.fc
        # We will modify directly that module instead of creating a new one as a new variable
        # NOTE: It's important to follow this structure. This is because PyTorch stores the models' weights under a named dictionary.
        # If you define more modules with other names you will incur in an error when trying to load the weights of the pretrained model
        # later in the notebook.
        self.classification_layer = nn.Sequential(
            self.convnet.fc,  # Linear(ResNet output, 4*hidden_dim)
            nn.ReLU(inplace=True),
            nn.Linear(4*hidden_dim, hidden_dim)
        )

    def forward(self, x):
        # Encode all images
        feats = self.convnet(x)
        return feats

#### Model training

In [6]:
# Define training hyperparameters
batch_size=64
hidden_dim=128
temperature=0.07
weight_decay=1e-4
max_epochs=500
lr=2e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'

epochs = 10
maxit = 10

# Define SimCLR Model
simclr_model = SimCLR(hidden_dim)

# Define Optimizer and Learning rate Scheduler
optimizer = AdamW(simclr_model.parameters(), lr=lr, weight_decay=weight_decay)
lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max = max_epochs, eta_min=lr/50)

# Training Loop
train_losses = []
val_losses = []
train_top5_accs = []
val_top5_accs = []

for epoch in range(epochs):
    # Training iterations and update lists
    train_loss, train_top5_acc = train_contrastive_sup(simclr_model, device, dataset_csi_test, optimizer, lr_scheduler, epoch, loss_temperature=temperature)
    train_losses.append(train_loss)
    train_top5_accs.append(train_top5_acc)
    print()

    # Validation iterations
    val_loss, val_top5_acc = valid_constrastive_sup(simclr_model, device, dataset_csi_val, epoch, loss_temperature=temperature)
    val_losses.append(val_loss)
    val_top5_accs.append(val_top5_acc)
    print()

# Save the model
torch.save(simclr_model.state_dict(), os.path.join(CHECKPOINT_PATH, 'My_SimCLR.ckpt'))

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[128, 1, 340, 100] to have 3 channels, but got 1 channels instead

## Ratatuja

In [7]:
device = 'cpu'
model = SimCLR(128)

for i, batch in enumerate(dataset_csi_train):
    batch_x, batch_y = batch
    print('start shape:',batch_x.shape)

    # Here the input is different from the self supervised case: batch_x is not a list of two,
    # rather a unique tensor with four channels. Therefore it has to be reshaped 
    # TO BE CHECKED
    
    batch_x_resh = batch_x.reshape(-1, 1, 340, 100)
    print('after reshaping:', batch_x_resh.shape)

    if i>= 3: break

    # Compute the features
    #features = model(cat_imgs)

start shape: torch.Size([32, 4, 340, 100])
after reshaping: torch.Size([128, 1, 340, 100])
start shape: torch.Size([32, 4, 340, 100])
after reshaping: torch.Size([128, 1, 340, 100])
start shape: torch.Size([32, 4, 340, 100])
after reshaping: torch.Size([128, 1, 340, 100])
start shape: torch.Size([32, 4, 340, 100])
after reshaping: torch.Size([128, 1, 340, 100])


In [11]:
import torch
import torch.nn.functional as F

# Example feature batch
# 4*3
features_batch = torch.tensor([[1.0, 0.0, 0.0],
                               [0.0, 1.0, 0.0],
                               [0.0, 0.0, 1.0],
                               [1.0, 1.0, 0.0]])

temperature = 0.5

# Compute Cosine Similarity
cos_sim = F.cosine_similarity(features_batch[:, None, :], features_batch[None, :, :], dim=-1)
print("Cosine Similarity Matrix:")
print(cos_sim)

# Mask out cosine similarity to itself
self_mask = torch.eye(cos_sim.shape[0], dtype=torch.bool, device=cos_sim.device)
cos_sim.masked_fill_(self_mask, -9e15)
print("\nCosine Similarity Matrix after Masking Self-Similarity:")
print(cos_sim)

# Find the positive example mask
pos_mask = self_mask.roll(shifts=cos_sim.shape[0] // 2, dims=0)
print("\nPositive Example Mask:")
print(pos_mask)

# NT_Xent loss
cos_sim = cos_sim / temperature
nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
#nll = nll.mean()
print("\nNegative Log-Likelihood (NLL) Loss:")
print(nll)

# Get ranking position of positive example
comb_sim = torch.cat([cos_sim[pos_mask][:, None], cos_sim.masked_fill(pos_mask, -9e15)], dim=-1)
sim_argsort = comb_sim.argsort(dim=-1, descending=True).argmin(dim=-1)
print("\nRanking Position of Positive Example:")
print(sim_argsort)

acc_top1 = (sim_argsort == 0).float().mean()
acc_top5 = (sim_argsort < 5).float().mean()
print("\nTop-1 Accuracy:")
print(acc_top1)
print("\nTop-5 Accuracy:")
print(acc_top5)

Cosine Similarity Matrix:
tensor([[1.0000, 0.0000, 0.0000, 0.7071],
        [0.0000, 1.0000, 0.0000, 0.7071],
        [0.0000, 0.0000, 1.0000, 0.0000],
        [0.7071, 0.7071, 0.0000, 1.0000]])

Cosine Similarity Matrix after Masking Self-Similarity:
tensor([[-9.0000e+15,  0.0000e+00,  0.0000e+00,  7.0711e-01],
        [ 0.0000e+00, -9.0000e+15,  0.0000e+00,  7.0711e-01],
        [ 0.0000e+00,  0.0000e+00, -9.0000e+15,  0.0000e+00],
        [ 7.0711e-01,  7.0711e-01,  0.0000e+00, -9.0000e+15]])

Positive Example Mask:
tensor([[False, False,  True, False],
        [False, False, False,  True],
        [ True, False, False, False],
        [False,  True, False, False]])

Negative Log-Likelihood (NLL) Loss:
tensor([1.8105, 0.3962, 1.0986, 0.8079])

Ranking Position of Positive Example:
tensor([1, 0, 0, 0])

Top-1 Accuracy:
tensor(0.7500)

Top-5 Accuracy:
tensor(1.)
